# YOLO Model Retraining Notebook

Este notebook orquesta el flujo completo de reentrenamiento del modelo YOLO.  
Soporta **dos modos** según el tipo de datos disponibles:

| `DATA_MODE` | Datos de entrada | Función |
|---|---|---|
| `"labeled"` | LogoDet-3K (XML PASCAL VOC, pre-etiquetado) | `convert_logodet_dataset()` |
| `"raw"` | Imágenes crudas sin etiquetar | `prepare_raw_dataset()` |

Flujo:
1. Preparación del dataset → formato YOLO
2. Validación de la estructura
3. Entrenamiento del modelo
4. Validación y evaluación

## 1. Importar librerías y configuración

In [2]:
import sys
from pathlib import Path

# Añadir raíz del proyecto al path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Importar módulo de datos (no necesita torch/ultralytics)
from retrain.retrain_scripts import data_preparation
from retrain.retrain_scripts.utils import setup_logging

# Configurar logging
logger = setup_logging()
logger.info("Notebook de reentrenamiento iniciado")

2026-02-08 22:47:40,061 - Retrain - INFO - Notebook de reentrenamiento iniciado
2026-02-08 22:47:40,061 - Retrain - INFO - Notebook de reentrenamiento iniciado


## 2. Configuración de rutas y parámetros

In [3]:
# ============================
# MODO DE DATOS
# ============================
# "labeled"  → LogoDet-3K (pre-etiquetado, XML PASCAL VOC)
# "raw"      → Imágenes crudas sin etiquetar
DATA_MODE = "labeled"   # <-- Cambiar a "raw" si solo tienes imágenes

# ============================
# RUTAS
# ============================
# Flujo A: Dataset pre-etiquetado (LogoDet-3K)
logodet_source_path = project_root / 'data' / 'LogoDet-3K'

# Flujo B: Imágenes crudas
raw_images_path = project_root / 'data' / 'raw' / 'retrain_img'

# Destino común del dataset YOLO (se usa en ambos flujos)
yolo_dataset_path = project_root / 'data' / 'processed' / 'retrain_yolo'

# Modelos
models_path = project_root / 'models'
base_model_path = models_path / 'best.pt'
trained_model_path = models_path / 'retrain' / 'best_retrained.pt'
train_results_path = models_path / 'retrain' / 'train_results'

# ============================
# PARÁMETROS DE SPLIT
# ============================
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
# TEST_RATIO = 0.15 (calculado automáticamente)

# ============================
# PARÁMETROS DE ENTRENAMIENTO
# ============================
EPOCHS = 100
BATCH_SIZE = 16
IMG_SIZE = 640
DEVICE = 0        # GPU 0, usar 'cpu' si no hay GPU
PATIENCE = 20
CONFIDENCE = 0.5

# Clases
CLASS_NAMES = ['logo']

print(f"Modo de datos:   {DATA_MODE}")
print(f"Dataset YOLO:    {yolo_dataset_path}")
print(f"Modelo base:     {base_model_path}")
if DATA_MODE == "labeled":
    print(f"Origen (XML):    {logodet_source_path}")
    print(f"  Existe: {logodet_source_path.exists()}")
else:
    print(f"Origen (raw):    {raw_images_path}")
    print(f"  Existe: {raw_images_path.exists()}")
print(f"Existe modelo:   {base_model_path.exists()}")

Modo de datos:   labeled
Dataset YOLO:    c:\Users\Coder\Desktop\projects\Proyecto_XII_equipo_4\data\processed\retrain_yolo
Modelo base:     c:\Users\Coder\Desktop\projects\Proyecto_XII_equipo_4\models\best.pt
Origen (XML):    c:\Users\Coder\Desktop\projects\Proyecto_XII_equipo_4\data\LogoDet-3K
  Existe: True
Existe modelo:   True


## 3. Preparación del dataset

Según `DATA_MODE`:
- **"labeled"** → Convierte LogoDet-3K (XML PASCAL VOC) a formato YOLO
- **"raw"** → Copia imágenes crudas y crea labels vacíos (para etiquetar con LabelImg / Roboflow)

In [4]:
if DATA_MODE == "labeled":
    # Flujo A: LogoDet-3K pre-etiquetado → YOLO
    dataset_stats = data_preparation.convert_logodet_dataset(
        source_dir=str(logodet_source_path),
        dest_dir=str(yolo_dataset_path),
        train_ratio=TRAIN_RATIO,
        val_ratio=VAL_RATIO,
        copy_images=True,
        seed=42
    )

elif DATA_MODE == "raw":
    # Flujo B: Imágenes crudas sin etiquetar
    dataset_stats = data_preparation.prepare_raw_dataset(
        raw_images_path=str(raw_images_path),
        dest_dir=str(yolo_dataset_path),
        class_names=CLASS_NAMES,
        train_ratio=TRAIN_RATIO,
        val_ratio=VAL_RATIO,
        seed=42
    )

else:
    raise ValueError(f"DATA_MODE no válido: '{DATA_MODE}'. Usa 'labeled' o 'raw'.")

# Mostrar resultado
if dataset_stats.get('success'):
    print(f"\n✓ Dataset preparado ({DATA_MODE})")
    print(f"  Total imágenes: {dataset_stats.get('total_images', '?')}")
    if DATA_MODE == "labeled":
        print(f"  Total anotaciones: {dataset_stats.get('total_annotations', '?')}")
    else:
        print(f"  ⚠ Etiquetas vacías - necesitan anotación manual")
    print(f"  Train: {dataset_stats.get('train', '?')}")
    print(f"  Val:   {dataset_stats.get('val', '?')}")
    print(f"  Test:  {dataset_stats.get('test', '?')}")
else:
    print(f"✗ Error: {dataset_stats.get('error', 'desconocido')}")

2026-02-08 22:47:56,606 - Retrain - INFO - Iniciando conversión de LogoDet-3K a YOLO...
2026-02-08 22:47:56,606 - Retrain - INFO - Iniciando conversión de LogoDet-3K a YOLO...
2026-02-08 22:47:56,610 - Retrain - INFO - Estructura de directorios YOLO creada
2026-02-08 22:47:56,610 - Retrain - INFO - Estructura de directorios YOLO creada
2026-02-08 22:47:56,612 - Retrain - INFO - Buscando pares imagen/XML...
2026-02-08 22:47:56,612 - Retrain - INFO - Buscando pares imagen/XML...
2026-02-08 22:48:00,715 - Retrain - INFO - Encontradas 158654 imágenes con anotaciones
2026-02-08 22:48:00,715 - Retrain - INFO - Encontradas 158654 imágenes con anotaciones
2026-02-08 22:48:00,801 - Retrain - INFO - Split: Train=111057, Val=23798, Test=23799
2026-02-08 22:48:00,801 - Retrain - INFO - Split: Train=111057, Val=23798, Test=23799
2026-02-08 22:48:00,803 - Retrain - INFO - Procesando train...
2026-02-08 22:48:00,803 - Retrain - INFO - Procesando train...
2026-02-08 22:59:50,219 - Retrain - INFO -   ✓


✓ Dataset preparado (labeled)
  Total imágenes: 158654
  Total anotaciones: 194263
  Train: 111057
  Val:   23798
  Test:  23799


In [5]:
# Validar estructura del dataset
is_valid = data_preparation.validate_dataset(str(yolo_dataset_path))

if is_valid:
    print("\n✓ Dataset válido - estructura correcta")
    if DATA_MODE == "labeled":
        data_preparation.validate_conversion(str(yolo_dataset_path), num_samples=3)
    else:
        print("⚠ Recuerda: las etiquetas están vacías.")
        print("  Anótalas antes de entrenar (LabelImg, CVAT, Roboflow...).")
else:
    print("\n✗ Dataset inválido - revisa la preparación")

# Mostrar data.yaml
data_yaml_path = yolo_dataset_path / 'data.yaml'
if data_yaml_path.exists():
    print("\n--- data.yaml ---")
    print(data_yaml_path.read_text())

2026-02-08 23:20:10,315 - Retrain - INFO -   ✓ images/train: 111057 archivos
2026-02-08 23:20:10,315 - Retrain - INFO -   ✓ images/train: 111057 archivos
2026-02-08 23:20:10,399 - Retrain - INFO -   ✓ images/val: 23798 archivos
2026-02-08 23:20:10,399 - Retrain - INFO -   ✓ images/val: 23798 archivos
2026-02-08 23:20:10,883 - Retrain - INFO -   ✓ labels/train: 111057 archivos
2026-02-08 23:20:10,883 - Retrain - INFO -   ✓ labels/train: 111057 archivos
2026-02-08 23:20:10,992 - Retrain - INFO -   ✓ labels/val: 23798 archivos
2026-02-08 23:20:10,992 - Retrain - INFO -   ✓ labels/val: 23798 archivos
2026-02-08 23:20:10,993 - Retrain - INFO -   ✓ data.yaml encontrado
2026-02-08 23:20:10,993 - Retrain - INFO -   ✓ data.yaml encontrado
2026-02-08 23:20:10,995 - Retrain - INFO - ==================================================
2026-02-08 23:20:10,995 - Retrain - INFO - ==================================================
2026-02-08 23:20:10,997 - Retrain - INFO - MUESTRAS DE VERIFICACIÓN
2026


✓ Dataset válido - estructura correcta


2026-02-08 23:20:11,612 - Retrain - INFO - 
10 Cane_1.txt:
2026-02-08 23:20:11,612 - Retrain - INFO - 
10 Cane_1.txt:
2026-02-08 23:20:11,629 - Retrain - INFO -   Anotaciones: 1
2026-02-08 23:20:11,629 - Retrain - INFO -   Anotaciones: 1
2026-02-08 23:20:11,629 - Retrain - INFO -     0 0.502703 0.463928 0.978378 0.446894
2026-02-08 23:20:11,629 - Retrain - INFO -     0 0.502703 0.463928 0.978378 0.446894
2026-02-08 23:20:11,629 - Retrain - INFO -   Imagen existe: ✓
2026-02-08 23:20:11,629 - Retrain - INFO -   Imagen existe: ✓
2026-02-08 23:20:11,629 - Retrain - INFO - 
10 Cane_10.txt:
2026-02-08 23:20:11,629 - Retrain - INFO - 
10 Cane_10.txt:
2026-02-08 23:20:11,650 - Retrain - INFO -   Anotaciones: 1
2026-02-08 23:20:11,650 - Retrain - INFO -   Anotaciones: 1
2026-02-08 23:20:11,652 - Retrain - INFO -     0 0.495868 0.462626 0.980716 0.444444
2026-02-08 23:20:11,652 - Retrain - INFO -     0 0.495868 0.462626 0.980716 0.444444
2026-02-08 23:20:11,653 - Retrain - INFO -   Imagen existe


--- data.yaml ---
# YOLOv8 Dataset Configuration
# Generado por data_preparation.py

path: c:\Users\Coder\Desktop\projects\Proyecto_XII_equipo_4\data\processed\retrain_yolo
train: images/train
val: images/val
test: images/test

nc: 1

names:
  0: logo



## 4. Entrenamiento del modelo

In [ ]:
# Importar módulos de entrenamiento (requieren torch + ultralytics)
from retrain.retrain_scripts import model_training, model_validation
print("✓ torch + ultralytics cargados correctamente")

In [ ]:
# Ruta al data.yaml generado
data_yaml_path = yolo_dataset_path / 'data.yaml'

# Entrenar modelo
training_info = model_training.train_model(
    model_path=str(base_model_path),
    dataset_yaml=str(data_yaml_path),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch_size=BATCH_SIZE,
    device=DEVICE,
    patience=PATIENCE,
    output_dir=str(train_results_path)
)

if training_info['success']:
    print(f"✓ Entrenamiento completado")
    print(f"  Epochs: {training_info['epochs']}")
    print(f"  Batch size: {training_info['batch_size']}")
    print(f"  Resultados: {training_info['output_dir']}")
else:
    print(f"✗ Error en entrenamiento: {training_info['error']}")

## 5. Validación del modelo

In [ ]:
# Validar modelo entrenado
validation_info = model_validation.validate_model(
    model_path=str(trained_model_path),
    dataset_yaml=str(data_yaml_path),
    output_dir=str(train_results_path),
    imgsz=IMG_SIZE,
    device=DEVICE
)

if validation_info['success']:
    print(f"✓ Validación completada")
else:
    print(f"✗ Error en validación: {validation_info['error']}")

## 6. Evaluación en conjunto de test

In [ ]:
# Evaluar en imágenes de test
test_images_path = yolo_dataset_path / 'images' / 'test'

eval_info = model_validation.evaluate_model(
    model_path=str(trained_model_path),
    test_images_path=str(test_images_path),
    output_dir=str(train_results_path),
    confidence=CONFIDENCE
)

if eval_info['success']:
    print(f"✓ Evaluación completada")
    print(f"  Imágenes procesadas: {eval_info['test_images']}")
    print(f"  Confianza mínima: {eval_info['confidence']}")
else:
    print(f"✗ Error en evaluación: {eval_info['error']}")

## 7. Resumen del reentrenamiento

In [ ]:
print("=" * 60)
print("RESUMEN DEL REENTRENAMIENTO")
print("=" * 60)
print(f"\nModo de datos:    {DATA_MODE}")
if DATA_MODE == "labeled":
    print(f"Dataset origen:   {logodet_source_path}")
else:
    print(f"Dataset origen:   {raw_images_path}")
print(f"Dataset YOLO:     {yolo_dataset_path}")
print(f"Modelo base:      {base_model_path}")
print(f"Modelo entrenado: {trained_model_path}")
print(f"\nDatos:")
print(f"  Train/Val/Test: {TRAIN_RATIO}/{VAL_RATIO}/{1-TRAIN_RATIO-VAL_RATIO:.2f}")
if dataset_stats.get('success'):
    print(f"  Imágenes: {dataset_stats['total_images']}")
    if DATA_MODE == "labeled":
        print(f"  Anotaciones: {dataset_stats.get('total_annotations', 'N/A')}")
    else:
        print(f"  ⚠ Etiquetas: vacías (requieren anotación manual)")
print(f"\nEntrenamiento:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Image size: {IMG_SIZE}")
print(f"  Clases: {CLASS_NAMES}")
print(f"\nResultados en: {train_results_path}")
print("=" * 60)